In [46]:
import os
import json

import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
cgfp = pd.read_csv("~/Box/dsi-core/11th-hour/good-food-purchasing/CONFIDENTIAL_GFPP Product Attribute List_8.26.25.csv", dtype=str)

In [3]:
cgfp[["Product GTIN or UPC", "Vendor", "Brand Name", "Product Type", "Food Product Category", "Product Name", "Level of Processing"]]

,Product GTIN or UPC,Vendor,Brand Name,Product Type,Food Product Category,Product Name,Level of Processing
0,NaN,Sysco,NaN,FRUIT CUP PEACH COMDTY A422,Fruit,"fruit cup, peach, commodity",Ultra-Processed
1,NaN,Produce Express,NaN,ASPARAGUS 11# PENCIL (MEXICO),Vegetables,asparagus,Whole/Minimally Processed
2,NaN,Produce Express,NaN,"CELERY 24'S (SANTA MARIA, CA)",Vegetables,celery,Whole/Minimally Processed
3,NaN,Produce Express,NaN,ENGLISH CUCUMBER 12 COUNT,Vegetables,"cucumber, english",Whole/Minimally Processed
4,NaN,Produce Express,NaN,PERSIAN CUCUMBER 22#,Vegetables,"cucumber, persian",Whole/Minimally Processed
...,...,...,...,...,...,...,...
90482,NaN,Thurston Foods,UPSTATE FARMS,YOGURT VANILLA NF GF (3050000),Yogurt,"yogurt, flavored, nonfat",Ultra-Processed
90483,NaN,Thurston Foods,UPSTATE FARMS,YOGURT BLUEBERRY NF (3050400),Yogurt,"yogurt, flavored, nonfat",Ultra-Processed
90484,NaN,Thurston Foods,DANIMALS,YOGURT VANILLA DANIMAL NF GF (3051300),Yogurt,"yogurt, flavored, nonfat",Ultra-Processed
90485,NaN,Thurston Foods,UPSTATE FARMS,YOGURT CHERRY VANILLA NF GF (3051900),Yogurt,"yogurt, flavored, nonfat",Ultra-Processed


In [4]:
cgfp_with_upc = cgfp[cgfp["Product GTIN or UPC"].notna() & ~cgfp["Product GTIN or UPC"].isin(["#REF!"])].copy()
cgfp_with_upc["Product GTIN or UPC"] = cgfp_with_upc["Product GTIN or UPC"].apply(lambda x: f"{int(float(x.replace(' ', ''))):014d}")

In [37]:
cgfp_mini = cgfp_with_upc[["Product GTIN or UPC", "Vendor", "Brand Name", "Product Type", "Food Product Category", "Product Name", "Level of Processing"]]
cgfp_mini

,Product GTIN or UPC,Vendor,Brand Name,Product Type,Food Product Category,Product Name,Level of Processing
84,10731149515854,Sysco Hawaii,NaN,FISH SALMON CHUM FILLET SK/ON 2-3#,Fish (Wild),"salmon, wild",Whole/Minimally Processed
85,00044978102502,Sysco Hawaii,NaN,RICE JASMINE GOLDEN PHOENIX/CAMEL,Rice,"rice, jasmine",Whole/Minimally Processed
86,10027000383091,Sysco Hawaii,NaN,KETCHUP TOMATO POUCH PK 114Z NTF,Condiments & Snacks,"condiment, ketchup, ss",Ultra-Processed
87,10876681007822,Sysco Hawaii,NaN,FLATBREAD NAAN TEARDROP ORIG (6-8CT),Grain Products,"bread, flatbread",Ultra-Processed
88,10889356022952,Sysco Hawaii,NaN,OLIVES GREEN QUEEN STFD PIMENTO 100/120,Condiments & Snacks,"olive, stuffed",Ultra-Processed
...,...,...,...,...,...,...,...
89392,10078800113000,Performance Food Service,UPSTATE FARMS,YOGURT STRWBRY BLND NF,Yogurt,"yogurt, flavored",Ultra-Processed
89393,10070470311000,Performance Food Service,YOPLAIT,YOGURT TRIX TRIPLE CHRY TFF,Yogurt,"yogurt, flavored",Ultra-Processed
89394,10070470166000,Performance Food Service,YOPLAIT PARFAITPRO,YOGURT VANILLA LF POUCH TFF GF,Yogurt,"yogurt, flavored",Ultra-Processed
89395,10078800113000,Performance Food Service,UPSTATE FARMS,YOGURT VANILLA NF,Yogurt,"yogurt, flavored, nonfat",Ultra-Processed


In [6]:
off = pd.read_csv("~/Box/dsi-core/11th-hour/good-food-purchasing/en.openfoodfacts.org.products.csv.gz", sep="\t", dtype=str, usecols=range(60))
off["code"] = off["code"].apply(lambda x: f"{np.iinfo(np.uint64).max if len(x) >= 20 else np.uint64(x):014d}")

In [98]:
len(off)

4014198

In [35]:
off_mini = off[["code", "product_name", "brands", "categories", "ingredients_text", "nova_group"]]
off_mini.iloc[2000000:2000100]

,code,product_name,brands,categories,ingredients_text,nova_group
2000000,03760284500067,Crème glacée Praliné aux éclats d'amendes cara...,NaN,NaN,NaN,NaN
2000001,03760284700009,Baguette de pain,NaN,NaN,NaN,NaN
2000002,03760284700016,Baguette au seigle,NaN,NaN,NaN,NaN
2000003,03760284700030,Baguette complète bio,NaN,NaN,NaN,NaN
2000004,03760284700061,Pavé des copains Bio,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2000095,03760285379532,NaN,NaN,Bières IPA,NaN,NaN
2000096,03760285410013,Tuiles de sarrasin aux 2 graines de lin,NaN,NaN,NaN,NaN
2000097,03760285410020,Tuiles de sarrasin à l’algue sauvage de bretagne,NaN,NaN,NaN,NaN
2000098,03760285410037,Tuiles de sarrasin,NaN,Tuiles salées,NaN,NaN


In [26]:
merged = off_mini.merge(cgfp_mini, left_on="code", right_on="Product GTIN or UPC")

In [51]:
len(merged)

364

In [52]:
is_good_match = []
for i, (_, row) in enumerate(merged.iterrows()):
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {OPENAI_API_KEY}",
            },
            json={
                "model": "gpt-4.1",
                "messages": [
                    {"role": "system", "content": """
You are comparing food products from two databases. Return `{"is_good_match": true}`
if the food product described by the first group of data fields is like the same
food product as described by the second group.
""".strip()},
            {"role": "user", "content": f"""
First group:
  * product_name: {row['product_name']}
  * brands: {row['brands']}
  * categories: {row['categories']}
  * ingredients: {row['ingredients_text']}

Second group:
  * Vendor: {row['Vendor']}
  * Brand Name: {row['Brand Name']}
  * Product Type: {row['Product Type']}
  * Food Product Category: {row['Food Product Category']}
  * Product Name: {row['Product Name']}
""".strip()},
                ],
                "response_format": {
                    "type": "json_schema",
                    "json_schema": {
                        "name": "is_good_match",
                        "schema": {
                            "type": "object",
                            "properties": {
                                "is_good_match": {"type": "boolean"},
                            },
                            "required": ["is_good_match"],
                            "additionalProperties": False,
                        },
                    },
                },
            },
        )
        is_good_match.append(json.loads(response.json()["choices"][0]["message"]["content"])["is_good_match"])
    except Exception:
        is_good_match.append(None)
    print(f"{i} {row['product_name']} =?= {row['Brand Name']} ? {is_good_match[-1]}")

0 The Smartest Cookie =?= nan ? False
1 Hershey’s Syrup =?= nan ? False
2 nan =?= nan ? False
3 Vegan plus =?= nan ? False
4 Vegan plus =?= nan ? False
5 Nectar de cerise =?= nan ? False
6 Probiotic =?= nan ? False
7 Whey Protein - Chocolate =?= nan ? False
8 Skinny juice =?= nan ? False
9 No chicken kiyvs =?= nan ? False
10 McCroustillant bang bang - Mc Donalds =?= nan ? False
11 ISO whey zero =?= nan ? False
12 Miel de Forêt =?= nan ? False
13 Curry powder =?= nan ? False
14 Hennesy =?= nan ? False
15 Metaxa Private Reserve =?= nan ? False
16 nan =?= nan ? False
17 Apple Turnover =?= nan ? False
18 nan =?= nan ? False
19 ConfiNoix =?= nan ? True
20 Petit hoppelli noir =?= nan ? False
21 Mamma Mia  American BBQ =?= nan ? False
22 Fit Piggy Snickers Twist =?= nan ? False
23 Fit Piggy Brownie =?= nan ? False
24 Confiture de  chataigne Bio =?= nan ? False
25 RENE BENDEJA NUECES MACADAMIA =?= nan ? False
26 Rosmarin Kartoffeln Maggi =?= nan ? False
27 Organic Oats =?= nan ? False
28 Coop 

In [53]:
merged["is_good_match"] = is_good_match

In [93]:
merged["nova_group"] = merged["nova_group"].map({"1": 1, "2": 2, "3": 3, "4": 4})

In [79]:
merged["Level of Processing"] = merged["Level of Processing"].map({"Whole/Minimally Processed": 1, "Culinary Ingredient": 2, "Moderately Processed": 3, "Ultra-Processed": 4})

In [99]:
len(merged.query("is_good_match"))

286

The first four columns, `product_name`, `brands`, `categories`, `ingredients_list`, come from Open Food Facts.

The last five columns, `Vendor`, `Brand Name`, `Product Type`, `Food Product Category`, `Product Name`, come from CGFP.

They should agree because I'm filtering on ChatGPT's determination that they are the same food product.

In [80]:
merged.query("is_good_match")[["product_name", "brands", "categories", "ingredients_text", "Vendor", "Brand Name", "Product Type", "Food Product Category", "Product Name"]].iloc[60:120]

,product_name,brands,categories,ingredients_text,Vendor,Brand Name,Product Type,Food Product Category,Product Name
128,Cheez it grooves white cheddar peg bag,Sunshine,"Snacks,Salty snacks,Appetizers,Biscuits and cr...","Enriched flour (wheat flour, niacin, reduced i...",HT Hackney Company,KELLANOVA,CHEEZ IT GROOVES WHITE CHED,Condiments & Snacks,"cracker, flavored"
129,Cheez it Bacon & Cheddar Cheese,Cheez It,"snacks, salty-snacks, appetizers, da:crackers","Enriched flour (wheat flour, niacin, reduced i...",HT Hackney Company,KELLANOVA,CHEEZ IT DUOZ BACON/CHEDDAR,Condiments & Snacks,"cracker, flavored"
130,Cheez It Extra Cheesy,Cheez It,NaN,NaN,HT Hackney Company,KELLANOVA,CHEEZ IT EXTRA CHEESY,Condiments & Snacks,"cracker, flavored"
131,Cheez it White Cheddar,Cheez It,"snacks, salty-snacks, sweet-snacks, appetizers...","Enriched flour (wheat flour, niacin, reduced i...",HT Hackney Company,KELLANOVA,CHEEZ IT WHITE CHEDDAR,Condiments & Snacks,"cracker, flavored"
132,Cheez-It Crackers Original,Cheez-It,"snacks, salty-snacks, sweet-snacks, appetizers...","enriched flour (wheat flour, niacin, reduced i...",HT Hackney Company,KELLANOVA,CHEEZ IT,Condiments & Snacks,"cracker, flavored"
133,Cheez It Original,Sunshine,"Snacks,Salty snacks,Appetizers,Crackers(Appeti...","Enriched flour (wheat flour, niacin, reduced i...",HT Hackney Company,KELLANOVA,CHEEZ IT BOX,Condiments & Snacks,"cracker, flavored"
134,Cheez It Hot & Spicy Baked,Cheez It,"snacks, salty-snacks, sweet-snacks, appetizers...","Enriched flour (wheat flour, niacin, reduced i...",HT Hackney Company,KELLANOVA,CHEEZ IT HOT & SPICY,Condiments & Snacks,"cracker, flavored"
135,"Very Vanilla Soymilk, Single Serve",Silk,"Vanilla soy-based drinks,Vanilla soy milks,Soy...","Soymilk (Filtered Water, Whole _Soybeans_), Ca...",Ace Endico,SILK,MILK SOY VERY VANILLA ASEPTIC (690799),Beverages,"plant milk, soy, flavored"
136,"Soymilk, Chocolate","Silk, Wwf Operating Company",NaN,"Soymilk (filtered water, whole soybeans), cane...",Performance Food Service,SILK,MILK SOY CHOC ASEPTIC SNG SERVE,Beverages,"plantmilk, soy, flavored"
137,Very Vanilla Soymilk,"Silk, Wwf Operating Company",NaN,"Soymilk (filtered water, whole soybeans), cane...",Performance Food Service,SILK,MILK SOY VERY VANILLA ASEPTIC,Beverages,"plantmilk, soy, flavored"


In [81]:
merged.query("is_good_match")[["product_name", "brands", "categories", "ingredients_text", "Vendor", "Brand Name", "Product Type", "Food Product Category", "Product Name"]].iloc[120:180]

,product_name,brands,categories,ingredients_text,Vendor,Brand Name,Product Type,Food Product Category,Product Name
191,pop. tart bites,NaN,NaN,NaN,HT Hackney Company,KELLANOVA,KELL POPTART BITES CONFETTI CK,Condiments & Snacks,"snack, poptart"
192,Pop-tarts Bites Frosted Strawberry peg bag,Pop-Tarts,NaN,NaN,HT Hackney Company,KELLANOVA,KELL POPTART BITES FRST STRBRY,Condiments & Snacks,"snack, poptart"
193,Pop tarts bites chocolately fudge peg bag,NaN,"Snacks, Desserts, Sweet snacks, Biscuits and c...",NaN,HT Hackney Company,KELLANOVA,KELL POPTART BITES CHOC FUDGE,Condiments & Snacks,"snack, poptart"
195,Nutrigrain Apple Cinnamon,Kellogg's,"Snacks, Sweet snacks, Bars, Cereal bars, en:bi...","CRUST (WHOLE GRAIN OATS, ENRICHED FLOUR (WHEAT...",HT Hackney Company,KELLANOVA,KELL NUTRI GRAIN BARS APPLE,Condiments & Snacks,"bar, nutrigrain"
196,Nutri Grain Strawberry,Kellogg's,en:breakfast-bar,"WHOLE GRAIN OATS, ENRICHED FLOUR (WHEAT FLOUR,...",HT Hackney Company,KELLANOVA,KELL NUTRI GRAIN BARS STRAWBRY,Condiments & Snacks,"bar, nutrigrain"
197,Frosted hot fudge sundae toaster pastries,Kellogg's,"Snacks, Sweet snacks, Biscuits and cakes, Bisc...","Enriched flour (wheat flour, niacin, reduced i...",HT Hackney Company,KELLANOVA,KELL P/TART SS HOT FDGE SUNDAE,Condiments & Snacks,"snack, poptart"
198,Breakfasttoaster pastries,NaN,NaN,NaN,Performance Food Service,POP-TARTS,PASTRY TOASTER CINNAMON FRSTD,Condiments & Snacks,"snack, poptart, whole grain rich"
199,Breakfasttoaster pastries,NaN,NaN,NaN,Performance Food Service,POP-TARTS,PASTRY STRWBRY FRSTD WHL GRAIN,Condiments & Snacks,"snack, poptart, whole grain rich"
200,Poptarts S'mores,Kellogg's,"Snacks, Sweet snacks, Biscuits and cakes, Bisc...","enriched flour (wheat flour, niacin, reduced i...",HT Hackney Company,KELLANOVA,KELL P/TART SS FR S'MORES,Condiments & Snacks,"snack, poptart"
202,Krave cereal with flavored center,Kellogg's,"Plant-based foods and beverages, Plant-based f...","Chocolate flavored filling (sugar, soybean oil...",HT Hackney Company,WK KELLOGG SALES LLC,KELL KRAVE CHOC IN A CUP,Grain Products,"cereal, corn"


In [82]:
merged.query("is_good_match")[["product_name", "brands", "categories", "ingredients_text", "Vendor", "Brand Name", "Product Type", "Food Product Category", "Product Name"]].iloc[180:240]

,product_name,brands,categories,ingredients_text,Vendor,Brand Name,Product Type,Food Product Category,Product Name
253,Easy Cheese Cheddar,Nabisco,"Quesos de vaca,Cheddar","WHEY, CANOLA OIL, MILK PROTEIN CONCENTRATE, CH...",HT Hackney Company,MONDELEZ GLOBAL/NABISCO COOKIE,KRAFT EASY CHEDDAR CHEESE,Cheese,"cheese, cheddar"
254,Nabisco Chips Ahoy! Mini Chewy,Nabisco,"Snacks, Sweet snacks, Biscuits and cakes, Bisc...","Unbleached enriched flour (wheat flour, niacin...",HT Hackney Company,MONDELEZ GLOBAL/NABISCO COOKIE,NBC CHEWY CHIPS AHOY MINI BB,Condiments & Snacks,cookie
255,Golden Oreo mini,Oreo,"Snacks, Sweet snacks, Biscuits and cakes, Bisc...","Unbleached enriched flour (wheat flour, niacin...",HT Hackney Company,MONDELEZ GLOBAL/NABISCO COOKIE,NBC MINI GOLDEN OREO BIG BAG,Condiments & Snacks,cookie
256,Original Hommus,Cedar's,NaN,NaN,Ace Endico,CEDAR'S,HUMMUS ORIGINAL 5LB S* (748379),Condiments & Snacks,"spread, hummus"
257,Buttermist Food Spray & Pan Releaser,NaN,NaN,NaN,Thurston Foods,NaN,SHRTNING PAN SPRAY BUTTERMIST,Condiments & Snacks,"butter, spray"
258,Cooked Rice,Golden Pheonix,NaN,NaN,Sysco Hawaii,NaN,RICE JASMINE GOLDEN PHOENIX/CAMEL,Rice,"rice, jasmine"
259,Sugar Packet,Domino,NaN,NaN,Thurston Foods,NaN,SUGAR PACKET - .1OZ,Condiments & Snacks,"sugar, ss"
260,Premium Pure Cane Granulated Sugar,Domino,undefined,SUGAR,Ace Endico,DOMINO,SUGAR GRANULATED (786843),Condiments & Snacks,sugar
261,Premium pure cane granulated sugar,Domino,"Sweeteners, Sugars",Sugar,Performance Food Service,DOMINO,SUGAR GRANULATED TFF,Condiments & Snacks,sugar
262,Egg nog,NaN,NaN,NaN,Cream-O-Land Dairy,NaN,HG CLOVERLAND EGG NOG,Beverages,"drink, eggnog"


In [83]:
merged.query("is_good_match")[["product_name", "brands", "categories", "ingredients_text", "Vendor", "Brand Name", "Product Type", "Food Product Category", "Product Name"]].iloc[240:300]

,product_name,brands,categories,ingredients_text,Vendor,Brand Name,Product Type,Food Product Category,Product Name
315,Whole Grain Spaghetti,Barilla,NaN,NaN,Ace Endico,BARILLA,PASTA SPAGHETTI WH GRAIN (817058),Grain Products,"pasta, whole grain rich"
316,Icy Lemonade,NaN,NaN,NaN,Cream-O-Land Dairy,NaN,HP ICY LEMONADE,Beverages,"drink, lemonade"
317,Light Multigrain Bread,Nature's Harvest,"Plant-based foods and beverages, Plant-based f...","Whole wheat flour, water, wheat gluten, cellul...",NaN,NaN,NHV LITE40 MLTGRN 20,Grain Products,"bread, multigrain"
319,Salted Butter,Grassland,"Dairies, Fats, Spreads, Spreadable fats, Anima...","pasteurized cream, salt, contains: milk,",Ace Endico,GRASSLAND,BUTTER SOLIDS SALTED AA (213541),Butter,"butter, salted"
320,Nerds gummy clusters peg bag,Nerds,NaN,"corn syrup, sugar, dextrose, modified food sta...",HT Hackney Company,FERRARA PAN CANDY CO,NERDS CLUSTERS MED PEG,Condiments & Snacks,candy
322,Big burrito,NaN,"Sandwiches, Burritos",s used for in advice can o calories a dayutes ...,HT Hackney Company,CAMINO REAL KITCHENS,TINA BIG BURRITO DELI BF/BEAN,Meals,"burrito, beef, bean"
323,Tina’s Big Burrito,NaN,NaN,"Water, Enriched Flour, Pinto Beans, Soybean Oi...",HT Hackney Company,CAMINO REAL KITCHENS,TINA BIG BURRITO DELI BEAN/CHS,Meals,"burrito, cheese, bean"
324,Big Burrito,NaN,NaN,NaN,HT Hackney Company,CAMINO REAL KITCHENS,TINA BIG BURRITO DELI RED HOT,Meals,"burrito, beef"
325,Hawaiian Sun,Hawaiian Sun Products Inc.,"Plant-based foods and beverages, Beverages, Pl...","Water, cane sugar, passion fruit juice concent...",Y Hata,NaN,"DRINK, ORANGE PASSION (22123)",Beverages,"drink, juice, orange, passionfruit"
326,Pass o guava Nectar,Hawaiian Sun Products Inc.,"Plant-based foods and beverages, Beverages, Pl...","Water, cane sugar, guava puree, passion fruit ...",Y Hata,NaN,"DRINK, PASS O GUAVA (22111)",Beverages,"drink, juice, guava, passionfruit"


In [97]:
merged.query("is_good_match").groupby(["nova_group", "Level of Processing"], dropna=False, as_index=False).size()

,nova_group,Level of Processing,size
0,1.0,1.0,20
1,1.0,2.0,1
2,1.0,4.0,1
3,2.0,2.0,4
4,3.0,1.0,2
5,3.0,2.0,2
6,3.0,3.0,5
7,3.0,4.0,3
8,4.0,1.0,1
9,4.0,2.0,1


The confusion matrix:

|          | OFF = 1 |   2 |   3 |   4 |   ? |
|--:|--:|--:|--:|--:|--:|
| **CGFP = 1** |  20 |   0 |   2 |   1 |  25 |
|        **2** |   1 |   4 |   2 |   1 |   7 |
|        **3** |   0 |   0 |   5 |   3 |   2 |
|        **4** |   1 |   0 |   3 | 151 |  56 |
|        **?** |   0 |   0 |   0 |   1 |   1 |

Accuracy (among non-NaN, the upper-left 4×4 square) = 180/194 = 92.8%.

Agreement is pretty good, though NOVA categories 2 and 3 are very unrepresented. Open Food Facts is much more likely to leave a NOVA group unspecified, and biased toward NOVA = 0 when it does so (taking the CGFP assignments as truth).